## REFORMAT DATASET
Reformatting the layout of the overall dataset in terms of column information, tupled statsitical data, and identifying the null values.

In [24]:
##Importing Libraries For Exploration
import pandas as pd

NaturalDisaster_Data = pd.read_csv("./Storm Dataset/storm2010.csv")
NaturalDisaster_Data.head(5)

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,201002,35488,205181,4,9.47,ENE,LEON SPGS,29.7375,-98.4926,2944250,9829556
1,201001,35529,205443,1,0.56,E,VICTORIA,35.1700,-85.5600,3510200,8533600
2,201001,35529,205444,1,0.89,SW,SOUTH PITTSBURG,35.0200,-85.7100,351200,8542600
3,201001,35529,205445,1,0.69,N,NEW SALEM,35.2400,-85.1500,3514400,859000
4,201001,35529,205446,1,1.38,S,RATHBURN,35.2600,-85.1700,3515600,8510200
